In [1]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional
import pandas as pd
from dotenv import load_dotenv
import os
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import Runnable


In [2]:
load_dotenv()


True

In [3]:
train_df = pd.read_csv("data/Train data.csv",low_memory=False)
bus_df = pd.read_csv("data/indian_bus_routes_large.csv")
plane_df = pd.read_csv("data/plane data.csv")

# Preview
print("Train:", train_df.shape)
print("Bus:", bus_df.shape)
print("Plane:", plane_df.shape)

Train: (186124, 4)
Bus: (10000, 4)
Plane: (300153, 4)


In [4]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0.2,
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model_name="llama3-70b-8192"
)


In [5]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

C:\Users\mayur.nikam\AppData\Local\Temp\ipykernel_24984\3353824168.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [6]:
class AgentState(TypedDict):
    user_input: str
    source: Optional[str]
    destination: Optional[str]
    preference: Optional[str]  # 'cost', 'time', or 'best'
    result: Optional[str]

In [7]:
def input_node(state: AgentState) -> AgentState:
    query = state["user_input"]
    chat_history = state.get("chat_history", [])

    # Check if user asked about memory
    if "last message" in query.lower():
        last = memory[-2] if len(memory) >= 2 else "No previous messages."
        memory.append(f"LLM: Your last message was: {last}")
        return {
            **state,
            "result": f"🧠 Your last message was: {last}",
        }
    
    
    prompt = f"""Extract the source city, destination city, and user preference 
(cost, time, or best) from this query: '{query}'. 
Return in the format: 
Source: <source> 
Destination: <destination> 
Preference: <cost/time/best>
"""
    response = llm.invoke(prompt).content.lower()

    def extract_value(label):
        for line in response.splitlines():
            if line.startswith(label.lower()):
                return line.split(":")[1].strip()
        return None

    return {
        **state,
        "source": extract_value("source"),
        "destination": extract_value("destination"),
        "preference": extract_value("preference"),
    }


In [8]:
def get_best_transport(state: AgentState) -> AgentState:
    source = state["source"]
    destination = state["destination"]
    preference = state["preference"] or "best"

    if not source or not destination:
        return {**state, "result": "Please specify both source and destination cities."}


    results = []

    def safe_get(row, col):
        return row[col] if col in row and pd.notna(row[col]) else float("inf")

    def match(df, mode, src_col, dst_col, cost_col, time_col):
        matches = df[
            (df[src_col].str.lower().str.strip() == source.lower().strip()) &
            (df[dst_col].str.lower().str.strip() == destination.lower().strip())
        ]
        print(f"{mode} Available Transports:", len(matches))  # DEBUG PRINT

        for _, row in matches.iterrows():
            cost = safe_get(row, cost_col)
            time = safe_get(row, time_col)
            if cost != float("inf") and time != float("inf"):
                results.append({
                    "mode": mode,
                    "cost": cost,
                    "time": time
                })

    # Apply to all modes
    match(bus_df, "Bus","Source", "Destination", "Cost", "Duration")
    match(train_df, "Train", "Source", "Destination", "Cost", "Duration")
    match(plane_df, "Plane", "Source", "Destination", "Cost", "Duration")

    if not results:
        return {**state, "result": "No available transport found for this route."}

    if preference == "cost":
        best = min(results, key=lambda x: x["cost"])
    elif preference == "time":
        best = min(results, key=lambda x: x["time"])
    else:
        best = min(results, key=lambda x: x["cost"] + x["time"])

    result_str = f"Best mode: {best['mode']} | Cost: {best['cost']} | Time: {best['time']}"
    return {**state, "result": result_str}


In [ ]:
def output_node(state: AgentState):
    # print("User Query:", state["user_input"])
    print("Source:", state["source"])
    print("Destination:", state["destination"])
    print("Preference:", state["preference"])
    print("Recommendation:", state["result"])
    print("✅ Recommendation:", state["result"])
    return state



In [10]:
graph = StateGraph(AgentState)
graph.add_node("input", input_node)
graph.add_node("recommend", get_best_transport)
graph.add_node("output", output_node)

graph.set_entry_point("input")
graph.add_conditional_edges("input", lambda state: END if "result" in state else "recommend")
graph.add_edge("recommend", "output")
graph.set_finish_point("output")

runnable = graph.compile()

In [11]:
config = {"configurable": {"thread_id": "1"}}

In [12]:
# from IPython.display import Image
# Image(runnable.get_graph().draw_mermaid_png())

In [13]:
runnable.invoke({
    "user_input": "I want to go delhi to mumbai, I want less time."
})

Bus Available Transports: 1
Train Available Transports: 0
Plane Available Transports: 15289
Source: delhi
Destination: mumbai
Preference: time
Recommendation: Best mode: Plane | Cost: 5955 | Time: 2.0
✅ Recommendation: Best mode: Plane | Cost: 5955 | Time: 2.0


{'user_input': 'I want to go delhi to mumbai, I want less time.',
 'source': 'delhi',
 'destination': 'mumbai',
 'preference': 'time',
 'result': 'Best mode: Plane | Cost: 5955 | Time: 2.0'}